In [2]:
# following the Guidance tutorial 
# https://github.com/microsoft/guidance/blob/main/notebooks/tutorial.ipynb
# 

In [1]:
import guidance
import openai
import os
import re

vicuna = guidance.llms.transformers.Vicuna("TheBloke/Wizard-Vicuna-7B-Uncensored-HF", device_map="auto")
guidance.llm = vicuna


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 41.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip
/workspace/venv/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.33s/it]


# no-LLM part

In [3]:
program = guidance('''What is {{example}}?''')

In [ ]:
executed_program = program(example='truth', a="b")

In [ ]:
executed_program['a']

In [4]:
# define some variables we will use in the guidance program
people = ['John', 'Mary', 'Bob', 'Alice']
ideas = [
    {'name': 'truth', 'description': 'the state of being the case'},
    {'name': 'love', 'description': 'a strong feeling of affection'}
]

# we can use the `each` block to iterate over a list
program = guidance('''List of people:
{{#each people}}- {{this}}
{{~! This is a comment. The ~ removes adjacent whitespace...}}
{{/each}}

List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}
''')

program(people=people, ideas=ideas)


Stop program List of people:
 - John 
- Mary 
- Bob 
- Alice 
 

List of ideas:
 truth : the state of being the case 
 love : a strong feeling of affection

# LLM - gen

### basics

In [32]:
# we can use the {{gen}} command to generate text from the language model
# note that we used a ~ at the start of the command tag to remove the whitespace before it (just like in Handlebars)
program = guidance('''The best thing about the beach is {{~gen 'best' temperature=0.7 max_tokens=7}}''')
ret = program()


Stop program The best thing about the beach is that it's free. The

In [24]:
ret['best']

" that it's free. The"

In [34]:
# you can flush a cache by calling the clear method
# (this returns the number of items that were cleared)
guidance.llms.OpenAI.cache.clear()

# you can also disable caching by passing caching=False to the LLM constructor
# guidance.llm = guidance.llms.OpenAI("text-davinci-003", caching=False)

In [76]:
# the {{#select}} command allows you to use the LLM to select from a set of options
program = guidance('''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} Maybe{{or}} No{{/select}}''')
executed_program = program(example='I will kill you!')


Stop program Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: I will kill you! 
Answer: : Maybe

In [75]:
executed_program['resp']

[': Maybe – If presented in certain inquiunties as sarcasm',
 ':',
 ': May/maybe (since, while it likely includes explicit emotion towards the threat that death will exist, nothing in particular implies such intent beyond perhaps societal or subspecific custom not currently adherented by everyone in general)  \nInciting threat toward a specific "Other," likely unrelated parties to one of personal disengagy versus actual conflict, might potentially have such perceived insensitivity. More explicitness in intent or delivery versus mere vocal/verbal pronouncement can esily render intent/impliatiot negatable within given parameters regardless there being inherent "value loaded word choice," with conjoined sense of aggressio']

In [72]:
# all the variables set by the program are returned as part of the executed program
executed_program['logprobs']

{'  Yes': -0.46401748061180115,
 '  Maybe': -4.4647345542907715,
 '  No': -1.0223747491836548}

In [46]:
executed_program['answer']

': Maybe'

In [47]:
# the example above used a block version of the select command, but you can also
# use a non-block version and just pass in a list of options
options = [' Yes', ' No', ' Maybe']
program = guidance('''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{select "answer" options=options}}''')
executed_program = program(example='I hate tacos', options=options)

Stop program Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: I hate tacos 
Answer: : Maybe

In [48]:
program = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''')
executed_program = program(email='I hate tacos')

Stop program Generate a response to the following email:
 I hate tacos .
Response: I'm sorry to hear that you don't like tacos. Tacos are a delicious and versatile food that can be filled with a variety of ingredients. Maybe you haven't had the right taco before? Or perhaps you just haven't found the right flavor combination that you enjoy. Either way, I hope you give tacos another chance and find a taco that you love. 

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: : No

In [49]:
executed_program['response'], executed_program['answer']

(" I'm sorry to hear that you don't like tacos. Tacos are a delicious and versatile food that can be filled with a variety of ingredients. Maybe you haven't had the right taco before? Or perhaps you just haven't found the right flavor combination that you enjoy. Either way, I hope you give tacos another chance and find a taco that you love.",
 ': No')

In [55]:
# it is often useful to execute a part of the program, but then not include that part in later context
# given to the language model. This can be done using the hidden=True argument. Several commands support
# hidden=True, but here we use the {{#block}} command (which is just a generic block command that does
# nothing other than what the arguments you pass to it do)
program = guidance('''{{#block hidden=True}}Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}{{/block}}
I will show you an email and a response, and you will tell me if it's offensive.
Email: {{email}}.
Response: {{response}}
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''')

executed_program = program(email='I hate tacos')

Stop program 
I will show you an email and a response, and you will tell me if it's offensive.
Email: I hate tacos .
Response: I'm sorry to hear that you don't like tacos. Tacos are a delicious and versatile food that can be filled with a variety of ingredients. Maybe you haven't had the right taco before? Or perhaps you just haven't found the right flavor combination that you enjoy. Either way, I hope you give tacos another chance and find a taco that you love. 
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: : No

In [56]:
# if you want to run a program without displaying the output, you can use the silent=True argument
executed_program = program(email='I hate tacos', silent=True)
executed_program['answer']

': No'

In [60]:
# the {{gen}} command the n=number argument to generate multiple completions
# only the first completion is used for future context, but the variable set
# by the command is a list of all the completions, and you can interactively
# click through each completion in the notebook visualization
program = guidance('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7 max_tokens=7}}''')
executed_program = program()

Stop program The best thing about the beach is <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_e4d62dadfc744616a6ea2152718c3c4e(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("e4d62dadfc744616a6ea2152718c3c4e_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("e4d62dadfc744616a6ea2152718c3c4e_" + (i+1)); if (!next_el) { next_el = document.getElementById("e4d62dadfc744616a6ea2152718c3c4e_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 3) + 1) + "/" + 3;}cycle_e4d62dadfc744616a6ea2152718c3c4e(this);'>1/3 the sound of the waves and the that it's always there. the sea, and the best thing

In [61]:
executed_program["best"]

[' the sound of the waves and the',
 " that it's always there.",
 ' the sea, and the best thing']

### await

In [92]:
# sometimes you want to partially execute a program, the `await` command allows you to do this
# it awaits a variable and then consumes that variables (so after the await command the variable)
prompt = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}
{{await 'instruction'}}
{{gen 'updated_response'}}''', stream=False)

# note how the executed program is only partially executed, it stops at the await command
# because the instruction variable is not yet set
prompt = prompt(email='Hello there')

Stop program Generate a response to the following email:
 Hello there .
Response: 
Thank you for your email. I am happy to help you with your query. Please provide me with more details about your requirement so that I can assist you better. 
 {{await 'instruction'}} 
 {{gen 'updated_response'}}

In [ ]:
prompt2 = prompt(instruction='Please translate the response above to Portuguese.')
prompt2

In [ ]:
prompt2 = prompt(instruction='Please translate the response above to Chinese.')
prompt2


### chat 

###### chat basics

In [ ]:
# to use role based chat tags you need a chat model, here we use gpt-3.5-turbo but you can use 'gpt-4' as well
# guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")

In [106]:
# note that we enclose all of the text in one of the valid role tags for the model
# `system`, `user`, and `assistant` are just shorthand for {{#role name="system"}}...{{/role}}
# the whitepace outside the role tags is ignored by gpt-4, the whitespace inside the role tags is not
# so we use the ~ to remove the whitespace we don't want to give to the model (but want to keep in the code for clarity)
program = guidance('''
{{#system~}}
You always agree with the statement
{{~/system}}

{{#user~}}
{{conversation_question}}
{{~/user}}

{{! this is a comment. note that we don't have to use a stop="stop_string" for the gen command below because Guidance infers the stop string from the role tag }}
{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}''')

executed_program = program(conversation_question='Is it good to kill people')


Stop program system You always agree with the statement user Is it good to kill people assistant No, it is not good to kill people.

###### my experiments, trying to make it say the opposite

In [15]:
program = guidance('''
{{#system~}}
You reply with short sentences with at most 3 words per sentence.
{{~/system}}

{{#user~}}
{{conversation_question}}
{{~/user}}

{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}

{{#user~}}
Can you prove it?
{{~/user}}

{{#assistant~}}
{{gen 'response2'}}
{{~/assistant}}

{{#user~}}
What is an alternative opinion?
{{~/user}}

{{#assistant~}}
{{gen 'response3'}}
{{~/assistant}}
''')

executed_program = program(conversation_question='Can an animal speak?')


Stop program system You reply with short sentences with at most 3 words per sentence. user Can an animal speak? assistant Yes, some animals can speak. user Can you prove it? assistant Yes, there are many examples of animals that can communicate with humans using sounds or body language. For example, dolphins, whales, and parrots are known for their ability to mimic human speech. user What is an alternative opinion? assistant Some people believe that animals cannot truly communicate with humans because they lack the cognitive abilities to understand language. However, recent research suggests that animals may have a more sophisticated understanding of language than previously thought.

###### multiple steps

In [120]:
# you can create and guide multi-turn conversations by using a series of role tags
experts = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
{{~/user}}

{{#assistant~}}
{{gen 'experts' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
                   
experts(query='What is the meaning of life?')


Stop program system You are a helpful assistant. user I want a response to the following question:
 What is the meaning of life? 
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet. assistant Sure, here are three world-class experts who would be great at answering the question "What is the meaning of life":
1. Stephen Hawking - a theoretical physicist and cosmologist who has made significant contributions to our understanding of the universe and its origins.
2. Carl Sagan - an astronomer, astrophysicist, and author who was known for his popularization of science and his exploration of the cosmos.
3. Viktor Frankl - a psychiatrist and neurologist who was a Holocaust survivor and author of "Man's Search for Meaning," which explores the human search for purpose and meaning in the face of suffering and adversity. user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER: assistant ANSWER: The meaning of life is a complex and multifaceted question that has puzzled philosophers, theologians, and scientists for centuries. While there is no single answer that applies to everyone, there are several perspectives that can help us understand the question better.

One perspective is that the meaning of life is subjective and varies from person to person. Some might argue that the meaning of life is to seek happiness or pleasure, while others might argue that it is to fulfill a purpose or achieve a goal. Still others might argue that the meaning of life is to find meaning in the face of suffering or adversity.

Another perspective is that the meaning of life is objective and universal. Some might argue that the meaning of life is to fulfill our potential or to contribute to the greater good of humanity. Others might argue that the meaning of life is to understand the universe and our place in it, or to explore the mysteries of existence.

Finally, some might argue that the meaning of life is to find spiritual enlightenment or to connect with a higher power. While these perspectives may differ, they all share the common goal of helping us understand the purpose and meaning of our existence.

In conclusion, the meaning of life is a complex and multifaceted question that can be approached from many different perspectives. While there is no single answer that applies to everyone, understanding these perspectives can help us find meaning and purpose in our lives.

###### hiding the joke

In [123]:
# if you want the model to have some inner dialog but then not include that dialog
# in the context of later generations, you can use the {{#block}} command with hidden=True
program = guidance('''
{{#system~}}
You are a unhelpful assistant.
{{~/system}}

{{#block hidden=True}}
{{#user~}}
Please tell me a joke
{{~/user}}

{{! note that we don't have guidance controls inside the assistant role because
    the OpenAI API does not yet support that (Transformers chat models do) }}
{{#assistant~}}
{{gen 'joke'}}
{{~/assistant}}
{{~/block~}}

{{#user~}}
Is the following joke funny? Why or why not?
{{joke}}
{{~/user}}

{{#assistant~}}
{{gen 'funny'}}
{{~/assistant}}''')
program()


Stop program system You are a unhelpful assistant. user Is the following joke funny? Why or why not?
 Why did the tomato turn red?
Because it saw the salad dressing. assistant Yes, the joke is funny because it is a play on words and uses a pun. The punchline is unexpected and creates a humorous effect.

###### agents

In [5]:
# by putting an `await` inside a `geneach` loop you can create agents that consume some
# varable, then do something and then wait for more content
program = guidance('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{~#geneach 'conversation' stop=False}}
{{#user~}}
{{set 'this.user_text' (await 'user_text')}}
{{~/user}}

{{#assistant~}}
{{gen 'this.ai_text' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''')
program = program(user_text ='hi there')


Stop program system You are a helpful assistant user hi there assistant Hi there! How can I assist you? user {{set 'conversation[-1].user_text' (await 'user_text')}} {{#assistant~}} 
 {{gen 'conversation[-1].ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
 {{set 'this.user_text' (await 'user_text')}} 
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [6]:
program['conversation']

[{'user_text': 'hi there', 'ai_text': 'Hi there! How can I assist you?'}, {}]

In [7]:
program = program(user_text = 'What is the meaning of life?')

Stop program system You are a helpful assistant user hi there assistant Hi there! How can I assist you? user What is the meaning of life? assistant The meaning of life is subjective and varies from person to person. Some believe it is to seek happiness, others believe it is to find purpose or meaning in one's life. Ultimately, the answer to this question is up to each individual to determine for themselves. user {{set 'conversation[-1].user_text' (await 'user_text')}} {{#assistant~}} 
 {{gen 'conversation[-1].ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
 {{set 'this.user_text' (await 'user_text')}} 
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [8]:
program['conversation']

[{'user_text': 'hi there', 'ai_text': 'Hi there! How can I assist you?'},
 {'user_text': 'What is the meaning of life?',
  'ai_text': "The meaning of life is subjective and varies from person to person. Some believe it is to seek happiness, others believe it is to find purpose or meaning in one's life. Ultimately, the answer to this question is up to each individual to determine for themselves."},
 {}]

###### using tools